In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/PS_20174392719_1491204439457_log.csv')

In [3]:
data.drop(['isFlaggedFraud'], axis=1, inplace=True)

In [4]:
data.head(20)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0


It would be helpful if this data had the timestamp when the transaction took place. This would help in determining the transaction time pattern per account.

The below column reference:

1. step: represents a unit of time where 1 step equals 1 hour
2. type: type of online transaction
3. amount: the amount of the transaction
4. nameOrig: customer starting the transaction
5. oldbalanceOrg: balance before the transaction
6. newbalanceOrig: balance after the transaction
7. nameDest: recipient of the transaction
8. oldbalanceDest: initial balance of recipient before the transaction
9. newbalanceDest: the new balance of recipient after the transaction
10. isFraud: fraud transaction

In [5]:
data.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
dtype: int64

In [6]:
data.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
dtype: object

## Feature engineering

In [7]:
name_type_amount_grp_df = data.groupby(['nameOrig', 'type'])['amount'].agg(['mean', 'min', 'max', 'count']).reset_index()

In [8]:
name_type_amount_grp_df.head()

,nameOrig,type,mean,min,max,count
0,C1000000639,CASH_OUT,244486.46,244486.46,244486.46,1
1,C1000001337,PAYMENT,3170.28,3170.28,3170.28,1
2,C1000001725,PAYMENT,8424.74,8424.74,8424.74,1
3,C1000002591,CASH_IN,261877.19,261877.19,261877.19,1
4,C1000003372,CASH_IN,20528.65,20528.65,20528.65,1


In [9]:
for ag in ['mean', 'min', 'max', 'count']:
    name_type_agg_grp_df = name_type_amount_grp_df.pivot(columns='type', index='nameOrig', values=ag).reset_index().fillna(0)
    name_type_agg_columns = name_type_agg_grp_df.columns
    name_type_agg_columns = ['nameOrig']+[col.lower()+f'_{ag}' for col in name_type_agg_columns if col!='nameOrig']
    name_type_agg_grp_df.columns = name_type_agg_columns
    data = data.merge(name_type_agg_grp_df, on='nameOrig', how='left')

In [10]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,cash_in_max,cash_out_max,debit_max,payment_max,transfer_max,cash_in_count,cash_out_count,debit_count,payment_count,transfer_count
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,...,0.0,0.0,0.0,9839.64,0.0,0.0,0.0,0.0,1.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,...,0.0,0.0,0.0,1864.28,0.0,0.0,0.0,0.0,1.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,...,0.0,0.0,0.0,0.00,181.0,0.0,0.0,0.0,0.0,1.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,...,0.0,181.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,...,0.0,0.0,0.0,11668.14,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
data.shape

(6362620, 30)